# Summary

- add summary template and additions to table
- add other adjustments as needed

# Outline
- 1: Sequence of Three Auto Functions that Inform Further Search
- 2: Look for Orbital Period with Periodograms
- 3: Look for Orbital Period with LC
- 4: Look for Super-Orbital Period with Periodograms
- 5: Look for Super-Orbital Period with LC

In [ ]:
from uncertainties import ufloat
from uncertainties.umath import *
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.stats import LombScargle
from scipy import signal
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import scipy.optimize
# from lmfit.models import GaussianModel
import glob
from astropy.table import Table,join,vstack,unique
from importlib import reload
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import ogle as o #module

blue = 'cornflowerblue'
navy = 'navy'
purple = 'rebeccapurple'
pink = '#CF6275'
maroon = 'maroon'

In [ ]:
#change src number here
num = 4
cross = Table.read('sourcefiles.csv')
full = Table.read('smc_x_m03_zar_match_OGLE_v2.ascii',format='ascii')
orb = float(cross[cross['src_n']==num]['Porb'])
orb

In [ ]:
full[full['src_n']==num]

- spin period: 

# 1: Sequence of Three Auto Functions that Inform Further Search

In [ ]:
ilist,vlist = o.getIV(num,cross,plot=True,zooms=True,figsize=(8,12),mult=(3,8),offset=10,stack=False) #sometimes good to adjust offset
iband,vband = vstack(ilist),vstack(vlist)

In [ ]:
idays = iband['MJD-50000']
imag = iband['I mag']
ierr = iband['I mag err']

In [ ]:
o.autopd(iband,orb,plotpd=True,plotphase=True,printall=False,ctime=True,orb_bounds=(10,20),cutlc=True,numcuts=10,plotdet=True,pbins=5,saveall=False,srcnum=num)

**confirmation of orbital pd already the 408 is probably the orbital period repeated/aliased**

In [ ]:
reload(o)
interp = o.colormag(iband,vband,ctime=True,retint=True) 

In [ ]:
#V-I vs time over I mag lc
fig,ax = plt.subplots(2,1,figsize=(8,8),sharex=True)
ax[0].scatter(vband['MJD-50000'],vband['V mag']-interp,color='black')
# plt.colorbar(label=('I mag'))
ax[1].scatter(idays,imag,color=pink,s=6)
plt.subplots_adjust(hspace=0.05)
maxi,mini = np.max(iband),np.min(imag)
ax[1].set_ylim(maxi+.03,mini-.03)

# 2: Look for Orbital Period with Periodograms

by OGLE epoch then dense regions (but can be broken up by >20 days)

In [ ]:
len(ilist)

In [ ]:
#without detrending
#always get high peak at exactly one day b/c of observing
for i in ilist:
    df,pks = o.multiphase(i,orb=orb,dense=False,minp=5,maxp=120,plotpd=True)

both epochs close to known orbital period, and orbital period looks good in phase-folds
- next: searching closer to orbital period and detrending


In [ ]:
#detrending each OGLE epoch
#varies just a bit based on window
for i in ilist:
    o.knownorb(i,orb,lower=10,upper=10,window=51,cutdata=False,cut1=0,cut2=500,plotdet=True,figsize=(10,5))


In [ ]:
orb

**separate into regions**

In [ ]:
#search and fold with densest region (max space up to 50 points)
df,pks = o.multiphase(iband,orb=orb,dense=True,maxspace=50,minp=5,maxp=100,plotpd=True)

In [ ]:
dense,mdense = o.finddense(iband,maxspace=50,retall=True)

In [ ]:
bps,maxpows,stdate,endate = o.denselcpd(iband,dense,maxp=100,plotbest=True,onlybp=True)

In [ ]:
bps

In [ ]:
#make tab for each year in LC
years = []
stdate = iband['MJD-50000'][0]
endate = iband['MJD-50000'][-1]
y = 1
while y < int((endate-stdate)/365)+1:
    #less than next year
    year = iband[iband['MJD-50000']<stdate+365*y]
    #also more than previous
    year = year[year['MJD-50000']>stdate+365*(y-1)]

    years.append(year)
    y+=1

In [ ]:
years[-1]['MJD-50000'][-1]

In [ ]:
len(years)

In [ ]:
#with multiphase big cell but nice to see what fold of known orb looks like even when it's not a peak
reload(o)
#adjust figsize as needed
fig = plt.figure(figsize=(22,10))
bps = []
p = 1
for y in years:
    freq,power,bp = o.periodogram(y,maxp=100,more=True,plot=False)
    bps.append(bp)
    ax = fig.add_subplot(3,6,p)
    ax.plot(1/freq,power,color='black')
    ax.axvline(orb,color='darkseagreen',alpha=0.5)
    p+=1

In [ ]:
#repeat with detrending -- can adjust window size and then more years will have enough points
reload(o)
fig = plt.figure(figsize=(22,10))
bps = []
p = 1
for y in years:
    if len(y)>99:
        o.detrend(y,window=99)
        freq,power,bp = o.periodogram(y,maxp=100,more=True,plot=False,det=True)
        bps.append(float(bp))
        ax = fig.add_subplot(3,6,p)
        ax.plot(1/freq,power,color='black')
        ax.axvline(orb,color='darkseagreen',alpha=0.5)
        p+=1

In [ ]:
bps

# 3: Look for Orbital Period with LC


In [ ]:
orb

In [ ]:
#identify reliable flare center to use for cen

def checkorb(st,end,cen=5797.4,orb=orb,plcen=False,figsize=(10,4)):
    stday = idays[st:st+1]
    enday = idays[end-1:end]
    fig = plt.figure(figsize=figsize)
    plt.errorbar(idays[st:end],imag[st:end],yerr=ierr[st:end],linestyle='none',marker='o',color=pink,markersize=5)
    maxi,mini = np.max(imag[st:end]),np.min(imag[st:end])
    plt.ylim(maxi+.02,mini-.02)

    fline = int((stday - cen)/orb)
    lline = int((enday - cen)/orb)
    for i in range(fline-1,lline+2):
        plt.axvline(cen+orb*i,alpha=0.2)
    if plcen: plt.axvline(cen,alpha=0.6)
checkorb(870,1000,plcen=True)

In [ ]:
def orbsub(inds,cen=5797.4,cenerr=0.4,pd=81.9,pderr=0.5,span=True,plcen=False,figsize=(22,30)):
    '''Separate full LC into subplots to better see orbital period timescale
    st,end are lists or arrays of start and end indices of each subplot
    cen: center of flare from which vertical lines are spaced (identify manually)
    add table/columns as arguments if moving to ogle.py'''
    sts = inds[:-1]
    ends = inds[1:]
    
    fig = plt.figure(figsize=figsize)
    rows = int(len(sts)/2)
    if len(sts)%2 == 1: rows+=1
    
    for i in range(len(sts)):
        st,end = sts[i],ends[i]
        #start at 1 for adding subplot
        ax = fig.add_subplot(rows,2,i+1)    
        tot = idays[end-1:end] - idays[st:st+1]
        stday = idays[st:st+1]
        enday = idays[end-1:end] 
    
        ax.errorbar(idays[st:end],imag[st:end],yerr=ierr[st:end],linestyle='none',marker='o',color=pink,markersize=5)
        maxi,mini = np.max(imag[st:end]),np.min(imag[st:end])
        ax.set_ylim(maxi+.02,mini-.02)
        if plcen: ax.axvline(cen)
        fline = int((stday - cen)/pd)
        lline = int((enday - cen)/pd)
        for i in range(fline-1,lline+1):
            #propogate error of ~3 days and 1 day on period
            if span and i<4: ax.axvspan(cen-cenerr+(pd-pderr)*i,cen+cenerr+(pd+pderr)*i,color='darkseagreen',alpha=0.4)
            ax.axvline(cen+pd*i,color='darkseagreen',alpha=0.6)

    return

In [ ]:
#list of inds to use -- easy to manipulate
#can also use finddense to not have large gaps, but then harder to control number of plots
inds = [0,79,181,284,359,517,714,800,900,1100,1177,1280,-1]
orbsub(inds)

In [ ]:
def checkpds(st,end,pd1=orb,pd2=orb+1,det=False,window=7,pbins=10):
    fig,ax = plt.subplots(1,2,figsize=(10,4),sharey=True)
    ttab = iband[st:end]
    if det:
        o.detrend(ttab,window=window)
        mag = ttab['I detrend']
    else: mag = ttab['I mag']
    days = ttab['MJD-50000']
    ax[0].scatter(days%pd1,mag,color=pink,label=str(pd1)+'d',s=6)
    ax[1].scatter(days%pd2,mag,color=pink,label=str(pd2)+'d',s=6)
    ax[0].scatter(pd1+days%pd1,mag,color=pink,s=6)
    ax[1].scatter(pd2+days%pd2,mag,color=pink,s=6)
    maxi,mini = np.max(mag[st:end]),np.min(mag[st:end])
    ax[0].set_ylim(maxi+.01,mini-.01)
    plt.subplots_adjust(wspace=0.05)
    ax[0].legend()
    ax[1].legend()
    
    mid,avg = o.meanphase(ttab,pd1,det=det,pbins=pbins)
    ax[0].plot(mid,avg,color='black',label=str(pd)+'d')
    ax[0].plot(pd1+mid,avg,color='black')

    mid,avg = o.meanphase(ttab,pd2,det=det,pbins=pbins)
    ax[1].plot(mid,avg,color='black',label=str(pd2)+'d')
    ax[1].plot(pd2+mid,avg,color='black')
checkpds(0,60)

# 4: Look for Super-Orbital Period with Periodograms

In [ ]:
#each OGLE epoch
for i in ilist:
    o.periodogram(i,maxp=1000)


In [ ]:
#full LC
df,pks = o.multiphase(iband,orb=orb,incl_orb=False,dense=False,minp=100,maxp=1000,plotpd=True)

# 5: Look for Super-Orbital Period with LC

In [ ]:
plt.figure(figsize=(20,5))
plt.scatter(idays,imag,color=pink,s=8)

maxi,mini = np.max(imag),np.min(imag)
plt.ylim(maxi+.05,mini-.05)

#add any known (Type II) outbursts